### Для входа нужны
- attend.csv
- groups.csv
- uniq_streets_coords.csv

### Выход
- _attend_features.csv

In [1]:
import numpy as np
import pandas as pd

### Читаем данные

In [2]:
df_attend = pd.read_csv('attend.csv')
display(df_attend)

,уникальный номер занятия,уникальный номер группы,уникальный номер участника,направление 2,направление 3,онлайн/офлайн,дата занятия,время начала занятия,время окончания занятия
0,401346550,801346550,101352023,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
1,401346550,801346550,101385462,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
2,401346550,801346550,101421897,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
3,401346550,801346550,101354499,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
4,401346550,801346550,101421312,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,2022-08-01,09:00:00,10:00:00
...,...,...,...,...,...,...,...,...,...
5901269,402103132,801371145,101421020,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00
5901270,402103132,801371145,101359314,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00
5901271,402103132,801371145,101357904,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00
5901272,402103132,801371145,101383123,ОНЛАЙН Пеший лекторий,ОНЛАЙН Краеведение и онлайн-экскурсии,Да,2023-01-31,12:30:00,14:30:00


### Группируем отдельные занятия посещения человеком, формируем новый столбец с количеством посещений

In [3]:
tmp = df_attend.groupby(['уникальный номер группы','уникальный номер участника'], as_index=False).size()
df_attend = tmp.merge(df_attend.drop_duplicates(subset=['уникальный номер группы', 'уникальный номер участника'], keep='last'), left_on=['уникальный номер группы','уникальный номер участника'], right_on=['уникальный номер группы','уникальный номер участника'], how='left')
df_attend = df_attend.drop(columns=['дата занятия','уникальный номер занятия'])
display(df_attend)

,уникальный номер группы,уникальный номер участника,size,направление 2,направление 3,онлайн/офлайн,время начала занятия,время окончания занятия
0,801346549,101370217,8,Здорово жить,Здорово жить,Нет,09:00:00,10:00:00
1,801346549,101375025,21,Здорово жить,Здорово жить,Нет,09:00:00,10:00:00
2,801346549,101392971,1,Здорово жить,Здорово жить,Нет,09:00:00,10:00:00
3,801346550,101352023,64,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,09:00:00,10:00:00
4,801346550,101354499,64,ОНЛАЙН Гимнастика,ОНЛАЙН Цигун,Да,09:00:00,10:00:00
...,...,...,...,...,...,...,...,...
600385,801373868,101374314,2,Художественно-прикладное творчество,"Керамика (глина, тестопластика)",Нет,13:00:00,14:00:00
600386,801373869,101372152,1,Рисование,Различные техники рисования,Нет,15:00:00,17:00:00
600387,801373869,101436322,1,Рисование,Различные техники рисования,Нет,15:00:00,17:00:00
600388,801373870,101439415,3,Домоводство,Вторая жизнь вещей,Нет,11:00:00,13:00:00


### Вычисляем длительность занятия

In [4]:
df_attend['lesson_length'] = (df_attend['время окончания занятия'].astype('datetime64') -df_attend['время начала занятия'].astype('datetime64')).astype('timedelta64[h]')

### Добавляем новое поле - часть дня занятия (утро, день, вечер)

In [5]:
b = [9,13,17,23]
l = ['mornig','aftennoon','evening']
df_attend['session'] = pd.cut(pd.to_datetime(df_attend['время начала занятия']).dt.hour, bins=b, labels=l, include_lowest=True)

### Удаляем ненужные столбцы

In [6]:
#df_attend = df_attend.drop(columns=['направление 2','направление 3','время начала занятия','время окончания занятия'])
df_attend = df_attend.drop(columns=['время начала занятия','время окончания занятия', 'направление 2'])

### Логарифмируем количество посещенией занятия юзером (уменьшаем разброс между большим и малым количеством занятий на разных курсах)

In [7]:
df_attend['size'] = np.log(df_attend['size'])

### Переименовываем стоблцы на английский для удобства

In [8]:
df_attend = df_attend.rename(columns={
    'size':'log_visits',
    "уникальный номер группы": "group_id",
    "уникальный номер участника": "user_id",
    'онлайн/офлайн':'is_online'})

In [9]:
df_attend.replace(('Да', 'Нет'), ('online', 'offline'), inplace=True)

## Обогащаем данными из таблицы с группами

In [10]:
df_groups = pd.read_csv('groups.csv')
df_groups = df_groups[['уникальный номер','направление 1','направление 2','адрес площадки']]
display(df_groups)

,уникальный номер,направление 1,направление 2,адрес площадки
0,801357270,Физическая активность,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2"
1,801356857,Физическая активность,ОФП,"город Москва, Подольская улица, дом 5"
2,801351684,Физическая активность,ОФП,"г. Москва, Базовская улица, дом 15, строение 1..."
3,801353683,Физическая активность,ОФП,"город Москва, улица Обручева, дом 28А, город М..."
4,801352164,Физическая активность,ОФП,"город Москва, Воронцовский парк, дом 3, город ..."
...,...,...,...,...
26998,801370473,Творчество,Красота и стиль,"город Москва, улица Маршала Катукова, дом 22"
26999,801370643,Игры,Шахматы и шашки,"город Москва, улица Маршала Катукова, дом 22"
27000,801370487,Физическая активность,ОФП,"город Москва, улица Маршала Катукова, дом 22"
27001,801372232,Физическая активность,"Фитнес, тренажеры","г. Москва, улица Свободы, дом 79"


### Вычисляем координаты занятия на основе адреса

In [11]:
# Выделяем улицу
def get_street(address):
    try:
        start = address.lower().find('москва')
        address = address[start+7:]
        address = address[:address.find(',')]
    except:
        address = 'москва'
    address = address.replace('улица','').strip().lower()
    return address

df_groups['user_address'] = df_groups['адрес площадки'].apply(get_street)



### Читаем таблицу улиц-координат

In [12]:
mid = '(55.7498, 37.5371)' # Башня Федерация

# Получаем координаты
df_uniq_streets_coords = pd.read_csv('uniq_streets_coords.csv')
df_uniq_streets_coords = df_uniq_streets_coords[['для координат','Coords']]
df_uniq_streets_coords['для координат']= df_uniq_streets_coords['для координат'].str.lower()
df_uniq_streets_coords['Coords'].fillna(mid, inplace=True)

streets_dict = dict(zip(df_uniq_streets_coords['для координат'], df_uniq_streets_coords['Coords']))

In [13]:
display(df_groups)

,уникальный номер,направление 1,направление 2,адрес площадки,user_address
0,801357270,Физическая активность,ОФП,"город Москва, Саратовская улица, дом 16, корпус 2",саратовская
1,801356857,Физическая активность,ОФП,"город Москва, Подольская улица, дом 5",подольская
2,801351684,Физическая активность,ОФП,"г. Москва, Базовская улица, дом 15, строение 1...",базовская
3,801353683,Физическая активность,ОФП,"город Москва, улица Обручева, дом 28А, город М...",обручева
4,801352164,Физическая активность,ОФП,"город Москва, Воронцовский парк, дом 3, город ...",воронцовский парк
...,...,...,...,...,...
26998,801370473,Творчество,Красота и стиль,"город Москва, улица Маршала Катукова, дом 22",маршала катукова
26999,801370643,Игры,Шахматы и шашки,"город Москва, улица Маршала Катукова, дом 22",маршала катукова
27000,801370487,Физическая активность,ОФП,"город Москва, улица Маршала Катукова, дом 22",маршала катукова
27001,801372232,Физическая активность,"Фитнес, тренажеры","г. Москва, улица Свободы, дом 79",свободы


### Добавляем координаты занятия (на основе адреса улицы)

In [14]:
def match_street_coords(street):  
    res = []
    for i in streets_dict.keys():
        if(i.find(street) != -1):
            res.append(streets_dict[i])    
    if len(res) > 0:
        return res[0]
    else:
        return '(55.7498, 37.5371)'
split_data = df_groups['user_address'].apply(match_street_coords).str.strip(')').str.strip('(').str.split(',')
df_groups['lat'] = split_data.apply(lambda x: x[0]).astype(float)
df_groups['long'] = split_data.apply(lambda x: x[1]).astype(float)
df_groups = df_groups.drop(columns=['адрес площадки'])
display(df_groups)

,уникальный номер,направление 1,направление 2,user_address,lat,long
0,801357270,Физическая активность,ОФП,саратовская,54.837868,39.231469
1,801356857,Физическая активность,ОФП,подольская,55.647600,37.710570
2,801351684,Физическая активность,ОФП,базовская,55.877819,37.511259
3,801353683,Физическая активность,ОФП,обручева,55.649562,37.564069
4,801352164,Физическая активность,ОФП,воронцовский парк,55.667631,37.534230
...,...,...,...,...,...,...
26998,801370473,Творчество,Красота и стиль,маршала катукова,54.853881,38.515771
26999,801370643,Игры,Шахматы и шашки,маршала катукова,54.853881,38.515771
27000,801370487,Физическая активность,ОФП,маршала катукова,54.853881,38.515771
27001,801372232,Физическая активность,"Фитнес, тренажеры",свободы,55.869949,37.443482


### Добавляем в attend столбцы направление 1, направление 2

In [15]:
df_attend = df_attend.merge(
    df_groups,
    left_on=['group_id'], 
    right_on=['уникальный номер'],
    how='left')
df_attend = df_attend.drop(columns=['уникальный номер'])

In [16]:
display(df_attend)

,group_id,user_id,log_visits,направление 3,is_online,lesson_length,session,направление 1,направление 2,user_address,lat,long
0,801346549,101370217,2.079442,Здорово жить,offline,1.0,mornig,Образование,Здорово жить,бульвар маршала рокоссовского,55.811285,37.725399
1,801346549,101375025,3.044522,Здорово жить,offline,1.0,mornig,Образование,Здорово жить,бульвар маршала рокоссовского,55.811285,37.725399
2,801346549,101392971,0.000000,Здорово жить,offline,1.0,mornig,Образование,Здорово жить,бульвар маршала рокоссовского,55.811285,37.725399
3,801346550,101352023,4.158883,ОНЛАЙН Цигун,online,1.0,mornig,Физическая активность,ОНЛАЙН Гимнастика,алтуфьевское шоссе,55.855709,37.584438
4,801346550,101354499,4.158883,ОНЛАЙН Цигун,online,1.0,mornig,Физическая активность,ОНЛАЙН Гимнастика,алтуфьевское шоссе,55.855709,37.584438
...,...,...,...,...,...,...,...,...,...,...,...,...
600385,801373868,101374314,0.693147,"Керамика (глина, тестопластика)",offline,1.0,mornig,Творчество,Художественно-прикладное творчество,лухмановская,55.717591,37.900113
600386,801373869,101372152,0.000000,Различные техники рисования,offline,2.0,aftennoon,Рисование,Рисование,большая очаковская,55.687729,37.463703
600387,801373869,101436322,0.000000,Различные техники рисования,offline,2.0,aftennoon,Рисование,Рисование,большая очаковская,55.687729,37.463703
600388,801373870,101439415,1.098612,Вторая жизнь вещей,offline,2.0,mornig,Творчество,Домоводство,большая очаковская,55.687729,37.463703


### Сохраняем обогащенную таблицу

In [17]:
df_attend.to_csv('_attend_features.csv')